# Introduction/Business Problem



## Reviewing Top 10 venues within 500 radius of Downtown Toronto.


### Café and Coffee shop are the top 10 venues for 28 neighbourhood with the freq between 0.2 to 0.19

### Café and Coffee shop are not listed within the top 10 venues for 8 neighbourhood


### Why ‘café’  is such populate venues in Christie with freq of 0.19 while Café in Church and Wellesley is only 0.02 freq ?

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import pandas as pd
dfge = pd.read_csv("https://cocl.us/Geospatial_data")
dfge.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
indexNames = df[ df['Borough'] == "Not assigned" ].index
df.drop(indexNames , inplace=True)
df['Neighbourhood'].loc[df['Neighbourhood'] == "Not assigned"] = df['Borough']
df.sort_values(by=['Postcode'])
#df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df = pd.merge(df[['Postcode','Borough','Neighbourhood']],
                 dfge[['Postcode','Latitude', 'Longitude']],
                 on='Postcode', 
                 how='left')
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
df.drop(columns=['Postcode'], axis=1, inplace=True)

neighborhoods_subset = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
neighborhoods_subset.head()


,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Regent Park,43.654260,-79.360636
2,Downtown Toronto,Ryerson,43.657162,-79.378937
3,Downtown Toronto,Garden District,43.657162,-79.378937
4,Downtown Toronto,St. James Town,43.651494,-79.375418


In [4]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version


In [5]:
neighborhood_name = neighborhoods_subset.loc[0, 'Neighborhood']
neighborhood_latitude = neighborhoods_subset.loc[0, 'Latitude'] 
neighborhood_longitude = neighborhoods_subset.loc[0, 'Longitude']

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()


In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
neighborhoods_subset_venues = getNearbyVenues(names=neighborhoods_subset['Neighborhood'],
                                   latitudes=neighborhoods_subset['Latitude'],
                                   longitudes=neighborhoods_subset['Longitude']
                                  )
#print(neighborhoods_subset_venues.shape)
#neighborhoods_subset_venues.head()
neighborhoods_subset_venues.groupby('Neighborhood').count()
#print('There are {} uniques categories.'.format(len(neighborhoods_subset_venues['Venue Category'].unique())))

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,15,15,15,15,15,15
Berczy Park,57,57,57,57,57,57
CN Tower,15,15,15,15,15,15
Cabbagetown,44,44,44,44,44,44
Central Bay Street,88,88,88,88,88,88
Chinatown,100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88


In [11]:
# one hot encoding
subset_onehot = pd.get_dummies(neighborhoods_subset_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
subset_onehot['Neighborhood'] = neighborhoods_subset_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [subset_onehot.columns[-1]] + list(subset_onehot.columns[:-1])
subset_onehot = subset_onehot[fixed_columns]

subset_grouped = subset_onehot.groupby('Neighborhood').mean().reset_index()


In [16]:
num_top_venues = 10

for hood in subset_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = subset_grouped[subset_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04
5               Bakery  0.03
6         Burger Joint  0.03
7                  Gym  0.03
8                Hotel  0.03
9                  Bar  0.03


----Bathurst Quay----
                venue  freq
0     Airport Service  0.20
1      Airport Lounge  0.13
2    Airport Terminal  0.13
3               Plane  0.07
4       Boat or Ferry  0.07
5             Airport  0.07
6  Airport Food Court  0.07
7        Airport Gate  0.07
8     Harbor / Marina  0.07
9    Sculpture Garden  0.07


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3          Steakhouse  0.04
4            Beer Bar  0.04
5         Cheese Shop  0.04
6  Seafood Restaurant  0.04
7              Bakery  0.04
8          Restaurant  0.04
9      Farmers Market  0.04




In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = subset_grouped['Neighborhood']

for ind in np.arange(subset_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(subset_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(36)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Hotel,Burger Joint,Bar,Bakery,Gym
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Plane,Boutique,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Gate,Airport
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Steakhouse,Café,Restaurant
3,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Plane,Boutique,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Gate,Airport
4,Cabbagetown,Coffee Shop,Italian Restaurant,Bakery,Market,Café,Pizza Place,Restaurant,Pub,Deli / Bodega,Beer Store
5,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant,Sushi Restaurant,Salad Place,Middle Eastern Restaurant,Japanese Restaurant,Restaurant
6,Chinatown,Café,Vegetarian / Vegan Restaurant,Bar,Dumpling Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Record Shop
7,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Convenience Store,Restaurant,Diner,Italian Restaurant,Baby Store
8,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Café,Mediterranean Restaurant,Bubble Tea Shop,Men's Store
9,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Bakery,Seafood Restaurant,Steakhouse,Italian Restaurant
